# Mass Spectrometry Group Alignment (GALAXY)

Anji Deng, Qihuang Zhang*

This tutorial demonstrates how to use **GALAXY** to align two MALDI mass spectrometry datasets
(e.g., Week 2 and Week 5 from the atherosclerosis regression experiment) and prepare them
for downstream joint analyses such as spatial segmentation.

We assume you have:

- Two CSV files containing the spectra (one row per pixel/spot, one column per m/z bin)  
- Two CSV files containing the spatial coordinates for each pixel/spot  
- The `MALDIpython` (GALAXY) package available in your Python path

---

## 0. Setup and package imports

Loading Packages

In [1]:
import os,csv,random
import pandas as pd
import numpy as np
import scanpy as sc
import math

from skimage import io, color
import torch


In [2]:
from scanpy import read_10x_h5
import SpaGCN as spg
import matplotlib.pyplot as plt
import json


In [3]:
from tqdm import tqdm
import pickle

In [5]:
import sys
sys.path.append('/lustre03/project/6075067/calcium/2021/MALDI/MALDI_package')

import MALDIpython as MALDIpy
MALDIpy.__version__

'1.0.0'

## 1. Load MALDI data and create AnnData objects

Replace the file paths and names below with your own.

In [6]:
weekpoint1 = "5 wk regression"
weekpoint2 = "2 wk regression"

groupshort = "pos"

if groupshort == "pos":
    group = "DHB pos"
else:
    group = "9AA neg"

DataDir = "data/Maaike"

In [7]:
DataDir

'data/Maaike'

In [8]:
weekpoint1

'5 wk regression'

In [9]:
group

'DHB pos'

In [10]:
MALDIdata1 = pd.read_csv("{DataDir}/{weekpoint} - {group} - All Spectra.csv".format(weekpoint = weekpoint1, DataDir = DataDir, group = group), sep =';') 
MALDIloc1 = pd.read_csv("{DataDir}/{weekpoint} - {group} - Region Spots.csv".format(weekpoint = weekpoint1, DataDir = DataDir, group = group), sep =';') 
MALDIdata2 = pd.read_csv("{DataDir}/{weekpoint} - {group} - All Spectra.csv".format(weekpoint = weekpoint2, DataDir = DataDir, group = group), sep =';') 
MALDIloc2 = pd.read_csv("{DataDir}/{weekpoint} - {group} - Region Spots.csv".format(weekpoint = weekpoint2, DataDir = DataDir, group = group), sep =';') 

In [11]:
mz_value1 = pd.DataFrame(list(MALDIdata1.columns[1:]), index = list(MALDIdata1.columns[1:])).astype('float')
mz_value1 = mz_value1.rename(columns = {0:"m/z"})

mz_value2 = pd.DataFrame(list(MALDIdata2.columns[1:]), index = list(MALDIdata2.columns[1:])).astype('float')
mz_value2 = mz_value2.rename(columns = {0:"m/z"})

In [12]:
mz_value1

,m/z
10,10.000000
10.126314163208,10.126314
10.252628326416,10.252628
10.378943443298,10.378943
10.505257606506,10.505258
...,...
1019.8834228516,1019.883423
1020.0097045898,1020.009705
1020.1360473633,1020.136047
1020.2623291016,1020.262329


In [15]:
np.diff(mz_value1["m/z"])

array([0.12631416, 0.12631416, 0.12631512, ..., 0.12634277, 0.12628174,
       0.12634277])

In [ ]:
mz_value2

Create AnnData object for each MSI data

In [35]:
MALDIdataAnn1 = sc.AnnData(X = MALDIdata1.iloc[:,1:], var = mz_value1, obs = MALDIloc1)
MALDIdataAnn2 = sc.AnnData(X = MALDIdata2.iloc[:,1:], var = mz_value2, obs = MALDIloc2)

/tmp/ipykernel_258932/895207559.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  MALDIdataAnn1 = sc.AnnData(X = MALDIdata1.iloc[:,1:], var = mz_value1, obs = MALDIloc1)
/home/calcium/.local/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/tmp/ipykernel_258932/895207559.py:2: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  MALDIdataAnn2 = sc.AnnData(X = MALDIdata2.iloc[:,1:], var = mz_value2, obs = MALDIloc2)


In [36]:
MALDIdataAnn1.obs["x"]

0          -5.817526
1           4.182474
2           4.182474
3           4.182474
4           4.182474
            ...     
18671    1694.182495
18672    1694.182495
18673    1694.182495
18674    1694.182495
18675    1694.182495
Name: x, Length: 18676, dtype: float64

## 2. Coordinate formatting and intensity normalization

GALAXY expects integer coordinates and benefits from per-spot normalization.

Transform the coordinates into integer, normalize the intensity value for each spot.

In [38]:
MALDIdataAnn1.obs = MALDIdataAnn1.obs.astype(int)
sc.pp.normalize_per_cell(MALDIdataAnn1)

In [39]:
MALDIdataAnn2.obs = MALDIdataAnn2.obs.astype(int)
sc.pp.normalize_per_cell(MALDIdataAnn2)

## 3. Peak calling and peak grouping

We use the `PeakCalling` function in GALAXY to identify peaks and group them.

In [64]:
PeakGroup= MALDIpy.PeakCalling (MALDIdataAnn1, MALDIdataAnn2)
PeakGroup

Use `.callpeak(threshold)` function to call the top threshold percent of peaks. Here we set the threshold as 0.9 to be unconservative

In [65]:
PeakGroup.callpeak(0.9)

Then, use `.grouppeaks(percentile)` to group the selected peaks into clusters. The `percentile` specified the percentile of the between-peak distances that can be a cutoff for different two clusters. Specifically, if two peaks have the distance greater than this cutoff, then these two peaks will regard as seperate groups. Otherwise, they will consider to be in the same group.

Typically, if the percentile is large, the more number of peaks in each cluster is expected.

In [66]:
PeakGroup.grouppeaks(0.9)

`grouppeaks(percentile)` function group the m/z values of the unknown spectrum and the reference spectrum separately. For every pair of clusters with one from unknown spectrum and another from the reference spectrum, if their m/z values have overlap, then they will be merged into one group, by taking the maximun and minimum of the m/z values of two groups.  For example,

In [67]:
PeakGroup.clusterUnk[0]

[23.01037979126, 23.136693954468, 23.263008117676]

In [68]:
PeakGroup.clusterRef[0]

[23.157243728638, 23.281370162964, 23.405494689941]

are merged as

In [69]:
PeakGroup.jointcluster[0]

[23.01037979126,
 23.136693954468,
 23.157243728638,
 23.263008117676,
 23.281370162964,
 23.405494689941]

Then we can use save_clusterresults to save the clusters into csv files.

In [70]:
MALDIpy.save_clusterresults (PeakGroup.clusterUnk, "output/example/clusterUnkranges")
MALDIpy.save_clusterresults (PeakGroup.clusterRef, "output/example/clusterRefranges")
MALDIpy.save_clusterresults (PeakGroup.jointcluster, "output/example/joint_combine_ranges")

## 4. Group alignment and fine alignment

Next, we build an alignment object and run the three main GALAXY steps:
correlation calculation, greedy group matching, and fine alignment.

First, create an object combining the unknown and reference spectra as a preparation for alignment.


In [71]:
ExactAlign = MALDIpy.AnnDataMALDI(MALDIdataAnn1, MALDIdataAnn2)

Calculate correlation matrix for each pair of groups (one from unknown and another from the reference spectrum).

In [72]:
ExactAlign.get_corr_peakgroup_refined(PeakGroup.jointcluster)

100%|██████████| 132/132 [00:06<00:00, 20.81it/s]


According to the calucated correlation matrix, perform a greedy algorithm to match two clusters

In [77]:
ExactAlign.greedy_match()

Finely align each pair of matched spectrum chunk by sliding both unknown and reference spectra. Identify the consensus optimal slide unit.

In [78]:
ExactAlign.fine_align(threshould = 0.2, ignore = True)

100%|██████████| 132/132 [00:00<00:00, 525.53it/s]


Finally, `summarize()` produce the aligned mz values.

In [80]:
ExactAlign.summarize()